# Deep learning LSTM code 

In [1]:
import numpy as np
import pandas as pd
import shutil
import skimage.io
from sklearn.ensemble import BaggingClassifier
import os
import imutils
from sklearn.metrics import confusion_matrix
import skimage.transform
from sklearn.neural_network import MLPClassifier
from os import listdir
from sklearn.ensemble import GradientBoostingClassifier
import sys
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import mahotas
import cv2
import os
import h5py
import matplotlib.pyplot as plt 
from skimage.feature import hog
import warnings
from matplotlib import pyplot
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score,f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.externals import joblib


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM,BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint

/opt/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Intialising variables

In [16]:
bins=8
seed=9
scoring="f1_macro"
num_trees=700

Readind dataset

In [17]:
train=pd.read_csv('train.csv')
validation=pd.read_csv('val.csv')
test=pd.read_csv('test_x.csv')

In [18]:
train_x=train.iloc[:,0:46]
train_y=train.iloc[:,46]
validation_x=validation.iloc[:,0:46]
validation_y=validation.iloc[:,46]

Data Balancing

In [5]:
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler
#smt = SMOTE(random_state=777, k_neighbors=1) #SMOTE
smt = ADASYN(random_state=42)                 #ADASYN
train_x_SMOTE, train_y_SMOTE = smt.fit_resample(train_x, train_y)
counts=np.unique(train_y_SMOTE, return_counts=True) 
print(counts)

Using TensorFlow backend.


(array([0, 1, 2]), array([30968, 30306, 30632]))


Feature Selection

In [6]:
#removing 0 variance features
# from sklearn.feature_selection import VarianceThreshold
# selector = VarianceThreshold()
# train_x=selector.fit_transform(train_x)
# validation_x=selector.transform(validation_x)
# test=selector.transform(test)

In [7]:
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2,mutual_info_classif,f_classif
from sklearn.decomposition import FactorAnalysis

select=SelectKBest(mutual_info_classif, k=40) #MI
# select=SelectKBest(chi2, k=30)                #CHI2
# select=SelectKBest(f_classif, k=30)           #f_classif
# select=FactorAnalysis(n_components=30, random_state=0)    #Factor Anayisi

train_x_SMOTE = select.fit_transform(train_x_SMOTE,train_y_SMOTE) 
validation_x= select.transform(validation_x) 
test=select.transform(test)

Running model

In [15]:
#on testing and validation
EPOCHS = 50  # how many passes through our data
BATCH_SIZE = 64  # how many batches? Try smaller batch if you're getting OOM (out of memory) errors.
labels=[0,1,2]

train_temp=train_x_SMOTE
train_temp=train_temp.reshape( len(train_temp),1, 40)
validation_temp=validation_x
validation_temp=validation_temp.reshape(len(validation_x),1 , 40)



model = Sequential()
model.add(LSTM(128, input_shape=(train_temp.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(3, activation='softmax'))


model.compile(optimizer="adam",loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(train_temp,train_y_SMOTE,epochs=EPOCHS,batch_size=BATCH_SIZE)

#preciting on validation dataset
y_pred_temp=model.predict(validation_temp)
y_pred=[labels[np.argmax(y_pred_temp[i])] for i in range(len(y_pred_temp))]
print("F1-macro",f1_score(validation_y, y_pred, average='macro') )


Epoch 1/50
1437/1437 [==============================] - 24s 17ms/step - loss: 1.0653 - accuracy: 0.4392
Epoch 2/50
1437/1437 [==============================] - 25s 17ms/step - loss: 1.0023 - accuracy: 0.4839
Epoch 3/50
1437/1437 [==============================] - 24s 17ms/step - loss: 0.9667 - accuracy: 0.5125
Epoch 4/50
1437/1437 [==============================] - 23s 16ms/step - loss: 0.9283 - accuracy: 0.5432
Epoch 5/50
1437/1437 [==============================] - 21s 15ms/step - loss: 0.8918 - accuracy: 0.5693
Epoch 6/50
1437/1437 [==============================] - 22s 15ms/step - loss: 0.8577 - accuracy: 0.5902
Epoch 7/50
1437/1437 [==============================] - 22s 15ms/step - loss: 0.8238 - accuracy: 0.6152
Epoch 8/50
1437/1437 [==============================] - 21s 15ms/step - loss: 0.7947 - accuracy: 0.6307
Epoch 9/50
1437/1437 [==============================] - 24s 17ms/step - loss: 0.7756 - accuracy: 0.6416
Epoch 10/50
1437/1437 [==============================] - 22s 16m

Code for test data

In [19]:
#combining train and  validation
train_x=train_x.append(validation_x)
train_y=train_y.append(validation_y)
counts=np.unique(train_y, return_counts=True) 
print(counts)

(array([0, 1, 2]), array([41283, 11325,  3089]))


In [20]:
# dataset balancing
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler
#smt = SMOTE(random_state=777, k_neighbors=1)
smt = ADASYN(random_state=42)
train_x_SMOTE, train_y_SMOTE = smt.fit_sample(train_x, train_y)
counts=np.unique(train_y_SMOTE, return_counts=True) 
print(counts)

(array([0, 1, 2]), array([41283, 39777, 40475]))


In [21]:
#features selection
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2,mutual_info_classif
select= SelectKBest(mutual_info_classif, k=40)
#select=FactorAnalysis(n_components=40, random_state=0)
train_x_SMOTE = select.fit_transform(train_x_SMOTE,train_y_SMOTE) 
validation_x= select.transform(validation_x) 
test=select.transform(test)

In [22]:
#model testing
#on testing
EPOCHS = 50  # how many passes through our data
BATCH_SIZE = 64  # how many batches? Try smaller batch if you're getting OOM (out of memory) errors.
labels=[0,1,2]

#train_temp=train_x_SMOTE.to_numpy()
train_temp=train_x_SMOTE.reshape( len(train_x_SMOTE),1, 40)
# validation_temp=validation_x.to_numpy()
# validation_temp=validation_temp.reshape(len(validation_x),1 , 46)
#test_temp=test.to_numpy()
test_temp=test.reshape(len(test),1 , 40)


model = Sequential()
model.add(LSTM(128, input_shape=(train_temp.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(3, activation='softmax'))


model.compile(optimizer="adam",loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(train_temp,train_y_SMOTE,epochs=EPOCHS,batch_size=BATCH_SIZE)


y_pred_temp=model.predict(test_temp)
y_pred=[labels[np.argmax(y_pred_temp[i])] for i in range(len(y_pred_temp))]
print("length of test",len(test))
print("length of reuslt:",len(y_pred))
counts = np.unique(y_pred, return_counts=True) 
print(counts)

Epoch 1/50
1899/1899 [==============================] - 34s 18ms/step - loss: 1.0535 - accuracy: 0.4463
Epoch 2/50
1899/1899 [==============================] - 32s 17ms/step - loss: 0.9926 - accuracy: 0.4936
Epoch 3/50
1899/1899 [==============================] - 30s 16ms/step - loss: 0.9566 - accuracy: 0.5215
Epoch 4/50
1899/1899 [==============================] - 31s 16ms/step - loss: 0.9180 - accuracy: 0.5479
Epoch 5/50
1899/1899 [==============================] - 30s 16ms/step - loss: 0.8821 - accuracy: 0.5756
Epoch 6/50
1899/1899 [==============================] - 30s 16ms/step - loss: 0.8509 - accuracy: 0.5961
Epoch 7/50
1899/1899 [==============================] - 31s 16ms/step - loss: 0.8225 - accuracy: 0.6144
Epoch 8/50
1899/1899 [==============================] - 30s 16ms/step - loss: 0.8024 - accuracy: 0.6268
Epoch 9/50
1899/1899 [==============================] - 30s 16ms/step - loss: 0.7811 - accuracy: 0.6393
Epoch 10/50
1899/1899 [==============================] - 30s 16m

In [109]:
# saving results
result=pd.DataFrame(columns=['row_id','relevance'])
temp=[i for i in range(len(test))]
result['row_id']=temp
result['relevance']=y_pred
result.to_csv('sample_submission.csv',index=False)